In [ ]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
import getpass
import os
os.environ["OPENAI_API_KEY"] = "sk-"

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader


In [ ]:
base_url = "http://127.0.0.1:9779/v1"
loader = TextLoader("./index.md")
docs = loader.load()


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
model_name = "/data/lixubin/models/m3e-base"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048)
chunked_docs = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    chunked_docs,
    hf
)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="/data/lixubin/models/Qwen/Qwen1.5-32B-Chat" ,base_url=base_url)

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


In [27]:
type(prompt)
print(prompt)

input_variables=['context', 'question'] metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [ ]:

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
type(rag_chain)

In [ ]:
rag_chain.invoke("MyGO是什么?")

In [ ]:
llm.invoke("做个自我介绍吧")

In [ ]:
# Get the prompt to use - you can modify this!
prompt_2 = hub.pull("hwchase17/self-ask-with-search")

In [ ]:
print(prompt_2.template)